### Autograd 

In [1]:
import math

def dz_dx(x):
    return 2*x*math.cos(x**2)

In [2]:
dz_dx(3)

-5.466781571308061

In [3]:
import torch

In [4]:
x = torch.tensor(3.0, requires_grad=True)

In [5]:
y = x **2

In [6]:
y

tensor(9., grad_fn=<PowBackward0>)

In [7]:
z = torch.sin(y)

In [8]:
z

tensor(0.4121, grad_fn=<SinBackward0>)

In [10]:
z.backward()

In [12]:
x.grad

tensor(-5.4668)

In [ ]:
y.grad # Refer pytorch documentations on directed acyclic graph (DAG)--> For intermediate tensors derivative not possible

/tmp/ipykernel_7351/486760323.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /pytorch/build/aten/src/ATen/core/TensorBody.h:489.)
  y.grad


### Long approach without Autograd

In [14]:
## Neural network (perceptron) 

import torch
x = torch.tensor(6.7) # Input feature 
y = torch.tensor(0.0) # True label (binary)

w = torch.tensor(1.0) # weight 
b = torch.tensor(0.0) # Bias 



In [15]:
## Binary cross entropy loss for scalar 

def binary_cross_entropy_loss(prediction, target):
    epsilon = 1e-8 # to prevent log(0)
    prediction = torch.clamp(prediction, epsilon, 1 - epsilon)
    return -(target * torch.log(prediction)+(1-target)*torch.log(1 - prediction))

In [16]:
## Forward pass 

z = w * x + b # Weighted sum linear part)
y_pred = torch.sigmoid(z) # predicted probability  

# Compute Binary cross-entropy loss 

loss = binary_cross_entropy_loss(y_pred, y)


In [17]:
loss

tensor(6.7012)

In [18]:
## Derivatives
# 1. dl/d(y-pred): (y_pred - y)/(y_pred*(1 - y_pred))
dloss_dy_pred = (y_pred - y)/(y_pred*(1 - y_pred))

#2. dy_pred/dz: Prediction (y_pred) with respect to z (sigmoid derivative)
dy_pred_dz = y_pred * (1 - y_pred)

# 3. dz/dw and dz/db: z with respect to w and b
dz_dw = x  # dz/dw = x
dz_db = 1  # dz/db = 1 (bias contributes directly to z)

dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

In [21]:
dloss_dy_pred, dy_pred_dz, dz_dw, dz_db

(tensor(813.3619), tensor(0.0012), tensor(6.7000), 1)

In [22]:
print(f"Manual Gradient of loss w.r.t weight (dw): {dL_dw}")
print(f"Manual Gradient of loss w.r.t bias (db): {dL_db}")

Manual Gradient of loss w.r.t weight (dw): 6.691762447357178
Manual Gradient of loss w.r.t bias (db): 0.998770534992218


### Autograd Approach 

In [23]:
x = torch.tensor(6.7)
y = torch.tensor(0.0)


In [36]:
w = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(0.0, requires_grad= True)
w,b

(tensor(1., requires_grad=True), tensor(0., requires_grad=True))

In [29]:
y_pred = torch.sigmoid(z)
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [34]:
loss = binary_cross_entropy_loss(y_pred, y)
loss

tensor(6.7012, grad_fn=<NegBackward0>)

In [32]:
loss.backward()

In [33]:
w.grad, b.grad

(tensor(6.6918), tensor(0.9988))

In [39]:
x  = torch.tensor([1.0, 2.0,3.0], requires_grad =True)
x

tensor([1., 2., 3.], requires_grad=True)

In [41]:
y = (x **2).mean()
y

tensor(4.6667, grad_fn=<MeanBackward0>)

In [42]:
y.backward()

In [43]:
x.grad

tensor([0.6667, 1.3333, 2.0000])

In [80]:
## Clearing Gradient
# Disable gradient tracking 

x = torch.tensor(2.0, requires_grad= True)
x

tensor(2., requires_grad=True)

In [81]:
y = x **2 
y

tensor(4., grad_fn=<PowBackward0>)

In [82]:
# Backward pass 

y.backward()


In [83]:
# Forward pass 

x.grad

tensor(4.)

In [76]:
## So, the problem here is gradients are getting accumulated 
# Here, the solution is we have to clear the gradient 

In [77]:
x.grad.zero_()

tensor(0.)

In [85]:
x.requires_grad_(False)
x

tensor(2.)

In [86]:
x

tensor(2.)

In [87]:
y = x ** 2

In [88]:
y

tensor(4.)

In [90]:
y.backward() # It won't work because we disable the grad function attribute 

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [92]:
# Using detatch()

x  = torch.tensor(2.0, requires_grad =True)
x

tensor(2., requires_grad=True)

In [93]:
z = x.detach()
z

tensor(2.)

In [94]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [95]:
y1 = z ** 2
y1

tensor(4.)

In [96]:
y.backward()

In [97]:
y1.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [98]:
## Using No_grad 

x  = torch.tensor(2.0, requires_grad =True)
x

tensor(2., requires_grad=True)

In [99]:
with torch.no_grad():
    y = x ** 2

In [100]:
y

tensor(4.)

In [101]:
y.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn